# Chapter 19: Advanced Graph Algorithms

> *"Beyond the basics lies a world of powerful techniques—network flows, matching, and decompositions—that unlock solutions to some of the most challenging problems in computer science."* — Anonymous

---

## 19.1 Introduction to Advanced Graph Algorithms

While fundamental graph algorithms like BFS, DFS, shortest paths, and MSTs solve a wide range of problems, many real-world applications require more sophisticated techniques. This chapter explores advanced graph algorithms that:

- **Model and optimize flow** through networks (transportation, communication, supply chains).
- **Find optimal matchings** (assignments, pairing).
- **Decompose trees** for efficient query processing.
- **Search heuristically** for optimal paths in large state spaces.

These algorithms are essential in operations research, bioinformatics, network design, and competitive programming.

---

## 19.2 Network Flow

Network flow problems deal with moving a commodity from sources to sinks through a network with capacity constraints. The **maximum flow problem** seeks to maximize the amount of flow, while **minimum cost flow** adds cost per unit.

### 19.2.1 Flow Networks: Definitions

A **flow network** is a directed graph G = (V, E) with:
- A **source** s (no incoming edges)
- A **sink** t (no outgoing edges)
- A **capacity** function c(u,v) ≥ 0 for each edge
- A **flow** f(u,v) satisfying:
  1. Capacity constraint: 0 ≤ f(u,v) ≤ c(u,v)
  2. Flow conservation: For all u ∈ V \ {s,t}, ∑_{v} f(v,u) = ∑_{v} f(u,v) (inflow = outflow)

The **value** of a flow is |f| = ∑_{v} f(s,v) - ∑_{v} f(v,s) (net outflow from source).

**Residual network:** For a given flow, the residual capacity is:
- Forward residual: c_f(u,v) = c(u,v) - f(u,v)
- Backward residual: c_f(v,u) = f(u,v) (allows "undoing" flow)

An **augmenting path** is a path from s to t in the residual network.

**Max-flow min-cut theorem:** The maximum flow value equals the capacity of the minimum s-t cut (partition of vertices with s in one side, t in the other, minimizing sum of capacities of edges crossing from source side to sink side).

### 19.2.2 Ford-Fulkerson Method

The Ford-Fulkerson method repeatedly finds augmenting paths and increases flow until no more exist.

```python
def ford_fulkerson(graph, source, sink):
    """
    graph: adjacency matrix or dict with capacities
    Returns max flow value.
    """
    n = len(graph)
    # Residual graph initially same as capacities
    residual = [row[:] for row in graph]
    parent = [-1] * n
    max_flow = 0

    def bfs(s, t):
        # Find augmenting path using BFS (Edmonds-Karp)
        visited = [False] * n
        queue = [s]
        visited[s] = True
        while queue:
            u = queue.pop(0)
            for v in range(n):
                if not visited[v] and residual[u][v] > 0:
                    parent[v] = u
                    if v == t:
                        return True
                    visited[v] = True
                    queue.append(v)
        return False

    while bfs(source, sink):
        # Find bottleneck capacity along path
        path_flow = float('inf')
        v = sink
        while v != source:
            u = parent[v]
            path_flow = min(path_flow, residual[u][v])
            v = u
        # Update residual network
        v = sink
        while v != source:
            u = parent[v]
            residual[u][v] -= path_flow
            residual[v][u] += path_flow
            v = u
        max_flow += path_flow
    return max_flow
```

**Time Complexity:** O(E * |f*|) where |f*| is max flow value. With integer capacities, it terminates but can be slow if capacities are large.

### 19.2.3 Edmonds-Karp Algorithm

Edmonds-Karp is the Ford-Fulkerson method with BFS for augmenting paths (as implemented above). BFS ensures each augmenting path is the shortest in terms of number of edges, leading to polynomial time.

**Complexity:** O(V E²) (each BFS O(E), number of augmentations O(VE)).

### 19.2.4 Dinic's Algorithm

Dinic's algorithm improves upon Edmonds-Karp by using level graphs and blocking flows. It works in phases: BFS to build level graph, then DFS to send blocking flow.

```python
from collections import deque

def dinic_max_flow(graph, source, sink):
    """
    graph: adjacency list with edges having 'capacity' attribute
    We'll represent graph as dict: graph[u] = [Edge(v, capacity, rev_index)]
    """
    n = len(graph)
    level = [-1] * n
    it = [0] * n
    
    class Edge:
        def __init__(self, to, rev, capacity):
            self.to = to
            self.rev = rev
            self.capacity = capacity
    
    # Build residual graph as adjacency list of Edge objects
    adj = [[] for _ in range(n)]
    for u in range(n):
        for v, cap in graph[u]:
            forward = Edge(v, len(adj[v]), cap)
            backward = Edge(u, len(adj[u]), 0)
            adj[u].append(forward)
            adj[v].append(backward)
    
    def bfs():
        for i in range(n):
            level[i] = -1
        queue = deque([source])
        level[source] = 0
        while queue:
            u = queue.popleft()
            for e in adj[u]:
                if e.capacity > 0 and level[e.to] < 0:
                    level[e.to] = level[u] + 1
                    queue.append(e.to)
        return level[sink] >= 0
    
    def dfs(u, flow):
        if u == sink:
            return flow
        for i in range(it[u], len(adj[u])):
            e = adj[u][i]
            if e.capacity > 0 and level[u] < level[e.to]:
                pushed = dfs(e.to, min(flow, e.capacity))
                if pushed > 0:
                    e.capacity -= pushed
                    adj[e.to][e.rev].capacity += pushed
                    return pushed
            it[u] += 1
        return 0
    
    max_flow = 0
    while bfs():
        it = [0] * n
        while True:
            pushed = dfs(source, float('inf'))
            if pushed == 0:
                break
            max_flow += pushed
    return max_flow
```

**Complexity:** O(V² E) worst-case, but often faster in practice. For unit capacity networks, O(min(V^(2/3), sqrt(E)) * E).

### 19.2.5 Applications of Max Flow

#### Maximum Bipartite Matching (as Flow)
Create a source connected to left set, edges from left to right with capacity 1, right to sink with capacity 1. Max flow equals maximum matching.

#### Minimum Cut
The min s-t cut corresponds to max flow (by max-flow min-cut theorem). Used in image segmentation (graph cuts), network reliability.

#### Edge-Disjoint Paths
Max flow gives maximum number of edge-disjoint paths from s to t (each edge capacity 1).

#### Circulation with Demands
Supply/demand problems can be modeled as flow with lower bounds.

---

## 19.3 Maximum Bipartite Matching

A **matching** in a graph is a set of edges without common vertices. In a bipartite graph (U ∪ V, E), a **maximum matching** pairs as many vertices as possible.

### 19.3.1 Hopcroft-Karp Algorithm

Hopcroft-Karp is the fastest known algorithm for maximum bipartite matching, running in O(E √V).

```python
from collections import deque

def hopcroft_karp(graph, U, V):
    """
    graph: adjacency list for vertices in U (0..U-1) to V (0..V-1)
    Returns: matching size and matching array for U (pair_U[u] = v or -1)
    """
    pair_U = [-1] * U
    pair_V = [-1] * V
    dist = [0] * U
    
    def bfs():
        q = deque()
        for u in range(U):
            if pair_U[u] == -1:
                dist[u] = 0
                q.append(u)
            else:
                dist[u] = float('inf')
        found = False
        while q:
            u = q.popleft()
            for v in graph[u]:
                pu = pair_V[v]
                if pu != -1 and dist[pu] == float('inf'):
                    dist[pu] = dist[u] + 1
                    q.append(pu)
                elif pu == -1:
                    found = True
        return found
    
    def dfs(u):
        for v in graph[u]:
            pu = pair_V[v]
            if pu == -1 or (dist[pu] == dist[u] + 1 and dfs(pu)):
                pair_U[u] = v
                pair_V[v] = u
                return True
        dist[u] = float('inf')
        return False
    
    matching = 0
    while bfs():
        for u in range(U):
            if pair_U[u] == -1 and dfs(u):
                matching += 1
    return matching, pair_U
```

**Algorithm phases:** BFS builds layers of unmatched vertices, DFS finds augmenting paths. Works in O(√V) phases.

---

## 19.4 Min-Cost Max-Flow

Given a flow network with costs per unit flow on edges, find a maximum flow of minimum total cost. This problem combines max flow with shortest paths.

### 19.4.1 Successive Shortest Path Algorithm

Repeatedly find cheapest augmenting path (in terms of cost) using potentials (Johnson's reweighting) to handle negative costs.

```python
import heapq

def min_cost_max_flow(graph, source, sink):
    """
    graph: adjacency list of (to, capacity, cost)
    Returns (max_flow, min_cost)
    """
    n = len(graph)
    # Build residual graph with Edge objects
    adj = [[] for _ in range(n)]
    for u in range(n):
        for v, cap, cost in graph[u]:
            forward = [v, cap, cost, len(adj[v])]
            backward = [u, 0, -cost, len(adj[u])]
            adj[u].append(forward)
            adj[v].append(backward)
    
    pot = [0] * n  # potentials for Johnson's
    prev_node = [0] * n
    prev_edge = [0] * n
    dist = [0] * n
    INF = float('inf')
    flow = 0
    flow_cost = 0
    
    while True:
        # Dijkstra with potentials
        for i in range(n):
            dist[i] = INF
        dist[source] = 0
        pq = [(0, source)]
        while pq:
            d, u = heapq.heappop(pq)
            if d != dist[u]:
                continue
            for i, e in enumerate(adj[u]):
                v, cap, cost, _ = e
                if cap > 0 and dist[v] > dist[u] + cost + pot[u] - pot[v]:
                    dist[v] = dist[u] + cost + pot[u] - pot[v]
                    prev_node[v] = u
                    prev_edge[v] = i
                    heapq.heappush(pq, (dist[v], v))
        if dist[sink] == INF:
            break
        # Update potentials
        for i in range(n):
            if dist[i] < INF:
                pot[i] += dist[i]
        # Find bottleneck
        aug_flow = INF
        v = sink
        while v != source:
            u = prev_node[v]
            e = adj[u][prev_edge[v]]
            aug_flow = min(aug_flow, e[1])
            v = u
        # Update residual graph
        v = sink
        while v != source:
            u = prev_node[v]
            e = adj[u][prev_edge[v]]
            e[1] -= aug_flow
            rev = adj[v][e[3]]
            rev[1] += aug_flow
            flow_cost += aug_flow * e[2]
            v = u
        flow += aug_flow
    return flow, flow_cost
```

**Complexity:** O(F * E log V) where F is max flow value. With potentials, Dijkstra runs in O(E log V) per augmentation.

### 19.4.2 Cycle Canceling Algorithm

Another approach: first find any max flow, then repeatedly find negative-cost cycles in the residual network and cancel them to reduce cost.

**Applications:** Transportation problems, assignment problems (with costs), minimum-cost matching.

---

## 19.5 Heavy-Light Decomposition (HLD)

Heavy-Light Decomposition is a technique to decompose a tree into paths (heavy paths) to answer path queries efficiently (e.g., maximum edge weight on path, sum of values, etc.) in O(log² n) time.

### 19.5.1 Definitions

- **Size** of a node: number of nodes in its subtree.
- **Heavy child:** The child with the largest subtree size (ties broken arbitrarily).
- **Light edge:** Edge from a node to a child that is not heavy.
- **Heavy path:** A maximal path where all edges are heavy.

HLD decomposes the tree into O(log n) heavy paths. Each node belongs to exactly one heavy path.

### 19.5.2 Decomposition Algorithm

```python
class HeavyLightDecomposition:
    def __init__(self, adj, root=0):
        self.n = len(adj)
        self.adj = adj
        self.parent = [-1] * self.n
        self.depth = [0] * self.n
        self.size = [1] * self.n
        self.heavy = [-1] * self.n  # heavy child
        self.head = [0] * self.n    # top of heavy path
        self.pos = [0] * self.n      # position in linear order (for segment tree)
        self.cur_pos = 0
        
        self.dfs_size(root, -1)
        self.decompose(root, -1, root)
    
    def dfs_size(self, u, p):
        self.parent[u] = p
        max_size = 0
        for v in self.adj[u]:
            if v == p:
                continue
            self.depth[v] = self.depth[u] + 1
            self.dfs_size(v, u)
            self.size[u] += self.size[v]
            if self.size[v] > max_size:
                max_size = self.size[v]
                self.heavy[u] = v
    
    def decompose(self, u, p, h):
        self.head[u] = h
        self.pos[u] = self.cur_pos
        self.cur_pos += 1
        if self.heavy[u] != -1:
            self.decompose(self.heavy[u], u, h)
        for v in self.adj[u]:
            if v != p and v != self.heavy[u]:
                self.decompose(v, u, v)
    
    def query_path(self, u, v):
        # Template: process path u-v using segment tree over pos
        res = 0  # or appropriate initial value
        while self.head[u] != self.head[v]:
            if self.depth[self.head[u]] < self.depth[self.head[v]]:
                u, v = v, u
            # query from pos[head[u]] to pos[u]
            # res = combine(res, seg_query(pos[head[u]], pos[u]))
            u = self.parent[self.head[u]]
        if self.depth[u] > self.depth[v]:
            u, v = v, u
        # now u is LCA, query from pos[u] to pos[v] (excluding u if edge values)
        # res = combine(res, seg_query(pos[u], pos[v]))
        return res
```

### 19.5.3 Applications

- **Path queries:** Max/min/sum on path between two nodes (using segment tree on `pos` array).
- **Path updates:** Add a value to all nodes on a path.
- **LCA queries:** HLD can also compute LCA by climbing paths.

**Complexity:** Each climb up a heavy path reduces depth, O(log n) climbs; each climb uses segment tree O(log n). Total O(log² n) per query.

---

## 19.6 Centroid Decomposition

Centroid decomposition recursively splits a tree at its centroid (node whose removal yields subtrees each of size ≤ n/2). This creates a decomposition tree of height O(log n) used for solving path counting problems.

### 19.6.1 Definition

The **centroid** of a tree is a node such that when removed, each resulting component has size ≤ n/2. Every tree has at least one centroid.

### 19.6.2 Decomposition Algorithm

```python
class CentroidDecomposition:
    def __init__(self, adj):
        self.adj = adj
        self.n = len(adj)
        self.deleted = [False] * self.n
        self.subtree_size = [0] * self.n
        self.parent_centroid = [-1] * self.n  # parent in centroid tree
    
    def dfs_size(self, u, p):
        self.subtree_size[u] = 1
        for v in self.adj[u]:
            if v != p and not self.deleted[v]:
                self.dfs_size(v, u)
                self.subtree_size[u] += self.subtree_size[v]
    
    def find_centroid(self, u, p, total_size):
        for v in self.adj[u]:
            if v != p and not self.deleted[v] and self.subtree_size[v] > total_size // 2:
                return self.find_centroid(v, u, total_size)
        return u
    
    def decompose(self, root, parent_centroid=-1):
        # Compute sizes
        self.dfs_size(root, -1)
        # Find centroid
        c = self.find_centroid(root, -1, self.subtree_size[root])
        self.parent_centroid[c] = parent_centroid
        # Mark centroid as deleted
        self.deleted[c] = True
        # Recurse on each component
        for v in self.adj[c]:
            if not self.deleted[v]:
                self.decompose(v, c)
        return c
    
    def build(self):
        return self.decompose(0)
```

### 19.6.3 Applications

- **Counting paths with constraints:** e.g., number of paths of length exactly k, paths with sum ≤ K, etc.
- **Divide-and-conquer on trees:** Solve problems by considering paths passing through centroid, then recurse.

**Complexity:** O(n log n) total if each level processes O(n) work.

---

## 19.7 A* Search Algorithm (Recap)

A* (A-star) is a heuristic search algorithm for finding the shortest path from a start node to a goal node. It was covered briefly in Chapter 17, but we expand here with emphasis on heuristics.

### 19.7.1 Algorithm

A* maintains for each node n:
- `g(n)`: actual cost from start to n
- `h(n)`: heuristic estimate of cost from n to goal
- `f(n) = g(n) + h(n)`

It expands nodes with smallest f, using a priority queue. The heuristic must be **admissible** (never overestimates true cost) for optimality, and **consistent** (or monotone) for efficiency.

### 19.7.2 Heuristics

Common heuristics:
- **Manhattan distance** for grid movement (4-direction).
- **Euclidean distance** for continuous space.
- **Octile distance** for 8-direction grids.
- **Pattern databases** for puzzles (15-puzzle).

### 19.7.3 Properties

- Optimal if heuristic is admissible.
- With consistent heuristic, A* never re-opens nodes.
- Time complexity: O(b^d) worst-case but often much better in practice.

---

## 19.8 Summary and Comparison

```
┌──────────────────────┬──────────────┬───────────────┬────────────────────────────┐
│ Algorithm            │ Time         │ Space         │ Use Case                   │
├──────────────────────┼──────────────┼───────────────┼────────────────────────────┤
│ Ford-Fulkerson       │ O(E|f|)      │ O(V+E)        │ Small capacities           │
│ Edmonds-Karp         │ O(VE²)       │ O(V+E)        │ General max flow           │
│ Dinic                │ O(V²E)       │ O(V+E)        │ Max flow, esp. unit caps   │
│ Hopcroft-Karp        │ O(E√V)       │ O(V+E)        │ Maximum bipartite matching │
│ Min-cost max-flow    │ O(F E log V) │ O(V+E)        │ Flow with costs            │
│ Heavy-Light Decomp    │ O(n) build   │ O(n)          │ Path queries on trees      │
│                      │ O(log² n)/q   │               │                            │
│ Centroid Decomp       │ O(n log n)   │ O(n)          │ Path counting on trees     │
└──────────────────────┴──────────────┴───────────────┴────────────────────────────┘
```

---

## 19.9 Practice Problems

### Problem 1: Maximum Flow (LeetCode 1514) – Actually LeetCode 1514 is "Path with Maximum Probability", not flow. For flow: LeetCode 1462? Better: Classic problems: "Maximum Flow" on UVA, SPOJ.

### Problem 2: Minimum Cost to Connect Two Groups of Points (LeetCode 1595)
Solve using min-cost max-flow.

### Problem 3: Bipartite Matching (LeetCode 1349) – Maximum Students Taking Exam.

### Problem 4: Path Queries on Tree (LeetCode 1483) – Kth Ancestor, can use HLD.

### Problem 5: Count Paths with Sum (LeetCode 437) – but on tree, can use centroid decomposition.

### Problem 6: Network Flow for Project Selection (max-weight closure)
Given projects with profits and prerequisites, find max profit subset.

### Problem 7: A* for Sliding Puzzle (LeetCode 773) – Sliding Puzzle, use Manhattan heuristic.

### Problem 8: Minimum Path Cover in DAG (can be reduced to max matching)

---

## 19.10 Further Reading

1. **"Introduction to Algorithms" (CLRS)** – Chapters 26 (Max Flow), 27 (Matching), 29 (Linear Programming)
2. **"Algorithm Design"** by Kleinberg & Tardos – Chapters 7 (Network Flow)
3. **"The Algorithm Design Manual"** by Steven Skiena – Chapter 8 (Network Flow)
4. **"Competitive Programming"** by Halim & Halim – Sections on flow, matching, tree decompositions
5. **Original Papers**:
   - Edmonds, J., & Karp, R. M. (1972) – "Theoretical improvements in algorithmic efficiency for network flow problems"
   - Dinic, E. A. (1970) – "Algorithm for solution of a problem of maximum flow in a network with power estimation"
   - Hopcroft, J. E., & Karp, R. M. (1973) – "An n^(5/2) algorithm for maximum matchings in bipartite graphs"
   - Sleator, D. D., & Tarjan, R. E. (1983) – "A data structure for dynamic trees" (related to HLD)

---

> **Coming in Chapter 20**: **Recursion and Backtracking** – We'll dive into recursive problem-solving, backtracking frameworks, and constraint satisfaction.

---

**End of Chapter 19**